# Ансамбли

В рамках этого семинара мы с вами попробуем различные алгоритмы ансамблирования для решения задачи классификации на [данных о транзакциях](https://www.kaggle.com/datasets/miznaaroob/fraudulent-transactions-data). Цель - определить, является ли транзакция мошеннической.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!gdown 1X11Qb_9opv2D3_1SjUOA5MZ7Sx84pT8o -O fraud.csv

In [ ]:
df = pd.read_csv('fraud.csv')

df.head()

In [ ]:
df.isna().any()

In [ ]:
df.isFraud.value_counts(normalize=True)

In [ ]:
X, y = df.drop(
    columns=['nameOrig', 'nameDest', 'isFlaggedFraud', 'isFraud']
).rename(
    columns={'oldbalanceOrg': 'oldbalanceOrig'}
), df.isFraud

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.65, random_state=42)
len(X_train), len(X_test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder(handle_unknown='ignore')
type_encoded_train = encoder.fit_transform(X_train['type'].to_numpy().reshape(-1, 1)).todense()
type_encoded_test = encoder.transform(X_test['type'].to_numpy().reshape(-1, 1)).todense()
feature_names = encoder.get_feature_names_out()

In [ ]:
type_encoded_train_df = pd.DataFrame(data=type_encoded_train, columns=feature_names)
type_encoded_test_df = pd.DataFrame(data=type_encoded_test, columns=feature_names)

type_encoded_train_df.index = X_train.index
type_encoded_test_df.index = X_test.index

X_train = pd.concat([X_train.drop(columns=['type']), type_encoded_train_df], axis=1)
X_test = pd.concat([X_test.drop(columns=['type']), type_encoded_test_df], axis=1)

## Baseline

Решим задачу базовыми алгоритмами:

1. Предсказанием самого популярного класса
2. Логистической регрессией
3. Решающим деревом

### Dummy-модель

In [ ]:
from sklearn.dummy import DummyClassifier


dummy = DummyClassifier()
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

Видим вполне предсказуемые результаты - доминантный класс предсказан хорошо, accuracy выглядит прилично, но метрики по минорному классу нулевые. Отсюда низкие усреднённые precision, recall и F1.

### Регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression


log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

Регрессия работает на этих данных, у нас неплохо подрос F1. Проверим, как справится дерево со стандартными параметрами.

### Дерево

In [ ]:
from sklearn.tree import DecisionTreeClassifier


tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

Дерево справилось отлично. Теперь будем сравнивать его эффективность с алгоритмами ансамблирования.

## Случайный лес

Построим для дерева график изменения F1-score в зависимости от глубины, т.к. хотим убедиться, что при той же глубине деревьев лес будет более устойчивым.

In [ ]:
# возьмем код с предыдущего семинара

import seaborn as sns
from tqdm import tqdm


def plot_fitting_curve(
    model_ctor, parameter: str, values: list, score, X_train, X_test, y_train, y_test
):
    train_curve = []
    test_curve = []
    for value in tqdm(values):
        model = model_ctor(**{parameter: value, 'random_state': 42})  # теперь создаем произвольную модель, конструктор которой нам был передан снаружи
        model.fit(X_train, y_train)
        y_pred_train, y_pred_test = model.predict(X_train), model.predict(X_test)
        train_curve.append(score(y_train, y_pred_train, average='macro'))
        test_curve.append(score(y_test, y_pred_test, average='macro'))
    sns.lineplot(x=values, y=train_curve)
    sns.lineplot(x=values, y=test_curve)

In [ ]:
from sklearn.metrics import f1_score


plot_fitting_curve(DecisionTreeClassifier, 'max_depth', np.arange(1, 20), f1_score, X_train, X_test, y_train, y_test)

Видим, что на отметке 10 дерево уже становится склонным к переобучению. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

plot_fitting_curve(RandomForestClassifier, 'max_depth', np.arange(6, 16, 2), f1_score, X_train, X_test, y_train, y_test)

Наша гипотеза подтвердилась, бэггинг на деревьях с глубиной дерева 12 ещё не переобучается. Видим, что лес является более устойчивой моделью.

In [ ]:
tree = DecisionTreeClassifier(max_depth=10, random_state=42)  # обучим "хорошее" дерево
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

In [ ]:
tree = DecisionTreeClassifier(max_depth=20, random_state=42)  # обучим дерево, склонное к переобучению
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

In [ ]:
forest = RandomForestClassifier(max_depth=20, random_state=42, n_jobs=-1)  # обучим аналогичный бэггинг
forest.fit(X_train, y_train)
y_pred = forest.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

Видим, что даже при большей глубине бэггинг является более устойчивым и показывает лучший результат.

## Стандартный бустинг

Посмотрим, как работает классификатор на бустинге. Важно: здесь мы будем всегда фиксировать значение параметра `random_state`, потому что сам по себе бустинг довольно неустойчив.

+ параметр verbose

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


vanilla_boosting = GradientBoostingClassifier(random_state=42)  # возьмем стандартные параметры
vanilla_boosting.fit(X_train, y_train)
y_pred = vanilla_boosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

Модель обучалась долго, но результат удивительно слабый. Что могло пойти не так?

Для начала попробуем ускорить обучение нашей модели. Для этого есть замечательный параметр `n_iter_no_change`, по сути выполняющий раннюю остановку обучения. Если добавление новых моделей не улучшает качество общей модели в течение нескольких итераций подряд, то мы завершаем обучение.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


vanilla_boosting = GradientBoostingClassifier(n_iter_no_change=5, random_state=42)
vanilla_boosting.fit(X_train, y_train)
y_pred = vanilla_boosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

Зафиксируем значение `n_iter_no_change = 5` и посмотрим, как ещё мы можем улучшить качество модели или ускорить обучение.

In [ ]:
# параметр subsample регулирует то, на какой части исходного датасета учится очередная модель

vanilla_boosting = GradientBoostingClassifier(subsample=0.8, n_iter_no_change=5, random_state=42)
vanilla_boosting.fit(X_train, y_train)
y_pred = vanilla_boosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

In [ ]:
# параметр learning_rate регулирует долю от предсказания очередной модели, которую мы прибавляем к суммарному ответу

vanilla_boosting = GradientBoostingClassifier(learning_rate=5e-2, n_iter_no_change=5, random_state=42)
vanilla_boosting.fit(X_train, y_train)
y_pred = vanilla_boosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

## XGBoost

XGBoost - библиотека с исходным кодом, в которой реализованы основные алгоритмы бустинга. Основная особенность XGBoost - параметры для регуляризации получившихся моделей:

* `booster`
* `reg_alpha`
* `reg_lambda`



In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgboosting = XGBClassifier(random_state=42)
xgboosting.fit(X_train, y_train)
y_pred = xgboosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

Рассмотрим подробнее параметр `booster`. Он может принимать следующие значения:

1. `gbtree` - обычный бустинг на деревьях
2. `gblinear` - бустинг на регрессиях
3. `dart` - бустинг на деревьях с дропаутом

dropout - удаление из модели части параметров для уменьшения емкости модели (одна из техник регуляризации)

In [ ]:
xgboosting = XGBClassifier(booster='dart', random_state=42)
xgboosting.fit(X_train, y_train)
y_pred = xgboosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

## LightGBM

LightGBM - библиотека с бустингами от Microsoft, идеологическое продолжение XGBoost. Основная особенность библиотеки - скорость. Модели содержат очень много настроек и кода для ускорения обучения и инференса.

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
lgboosting = LGBMClassifier(learning_rate=1e-2, random_state=42)
lgboosting.fit(X_train, y_train)
y_pred = lgboosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

## CatBoost

CatBoost - библиотека от Яндекса. Почти все делает из коробки, не требует перебора гиперпараметров. Всё, что нужно - подготовить достаточно чистые данные. За счет внутреннего подбора настроек работает дольше, чем XGBoost и LGBM.

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
catboosting = CatBoostClassifier(metric_period=50, random_state=42)
catboosting.fit(X_train, y_train)
y_pred = catboosting.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))